# COMP0173: Coursework 2

The paper HEARTS: A Holistic Framework for Explainable, Sustainable, and Robust Text Stereotype Detection by Theo King, Zekun Wu et al. (2024) presents a comprehensive approach to analysing and detecting stereotypes in text [1]. The authors introduce the HEARTS framework, which integrates model explainability, carbon-efficient training, and accurate evaluation across multiple bias-sensitive datasets. By using transformer-based models such as ALBERT-V2, BERT, and DistilBERT, this research project demonstrates that stereotype detection performance varies significantly across dataset sources, underlining the need for diverse evaluation benchmarks. The paper provides publicly available datasets and code [2], allowing full reproducibility and offering a standardised methodology for future research on bias and stereotype detection in Natural Language Processing (NLP).

# Instructions

All figures produced during this notebook are stored in the project’s `/COMP0173_Figures` directory.
The corresponding LaTeX-formatted performance comparison tables, including ALBERT-V2, BERT, and DistilBERT are stored in `/COMP0173_PDF`, with the compiled document available as `COMP0173-CW2-TABLES.pdf`.

# Technical Implementation (70%)

In [ ]:
# %%capture
# pip install -r requirements.txt
# pip install transformers
# pip install --upgrade transformers
# pip install --upgrade tokenizers
# pip install -U sentence-transformers
# pip install natasha
# pip install datasets
# pip install --user -U nltk
# conda install -c anaconda nltk
# pip install --upgrade openai pandas tqdm
# pip install dotenv
# python -m spacy download ru_core_news_lg

In [ ]:
# pip install -U pip setuptools wheel
# pip install -U spacy
# python -m spacy download en_core_web_trf
# python -m spacy download en_core_web_sm
# python -m spacy download ru_core_news_lg

# # GPU
# pip install -U 'spacy[cuda12x]'
# # GPU - Train Models
# pip install -U 'spacy[cuda12x,transformers,lookups]'

In [ ]:
# Import the libraries 
import random, numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(color_codes=True)
plt.style.use('seaborn-v0_8')

# To ignore warnings
import warnings
warnings.filterwarnings('ignore')
np.random.seed(23)

warnings.filterwarnings(
    "ignore",
    message="pkg_resources is deprecated as an API"
)

In [ ]:
# Import libraries 
import pandas as pd
import os
import sys
import importlib.util, pathlib
from pathlib import Path
import warnings 
from importlib import reload
from importlib.machinery import SourceFileLoader
from IPython.display import display
import pandas as pd
from pathlib import Path
import re
import difflib
import string
from collections import defaultdict
import json
import gc

In [ ]:

import torch
import transformers
from transformers import AutoModelForMaskedLM, XLMWithLMHeadModel
from transformers import AutoTokenizer, AutoConfig
from transformers import TrainingArguments, Trainer
from sentence_transformers import SentenceTransformer, util
import platform
from datasets import Dataset
# import spacy 
import requests
from tqdm import tqdm
import yaml
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
sys.path.append("Exploratory Data Analysis")
sys.path.append("Model Training and Evaluation")

from Initial_EDA import (
    prepare_target_variable_distribution,
    prepare_group_distribution,
    prepare_text_length_analysis,
    create_word_cloud
)

from Sentiment_Toxicity_Analysis_Ru import analyse_sentiment_and_regard

In [ ]:
# Check the GPU host (UCL access)
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0))

# # Path
# import os
# os.chdir("/tmp/HEARTS-Text-Stereotype-Detection")
# os.getcwd()

## Part 4: Adapt the model architecture and training pipeline to your local context

#### Helper Functions

In [ ]:
def pie_chart_domain(df, column, name):
    
    """
    Plot the percentage distribution of social-group domains as a styled pie chart.

    Parameters
    ----------
    df : pandas.DataFrame
        Input dataframe containing a categorical column representing social domains.
    column : str, optional
        Name of the column in `df` holding domain labels. 
        
    column : str, optional
        Name of the dataset. 

    Returns
    -------
    None
        Displays a pie chart visualising the proportional distribution of categories.
    
    Notes
    -----
    The function applies a custom colour palette tailored for the RuBias dataset 
    (gender, class, nationality, LGBTQ). Any unseen categories default to grey.
    """
    
    # Compute relative frequency (%) of categories
    domain_counts = df[column].value_counts(normalize=True) * 100
    labels = domain_counts.index
    sizes = domain_counts.values

    # Predefined colour palette
    color_map = {
        'gender':      "#CA5353",  
        'profession':  "#F1A72F",  
        'nationality': "#559A67",  
        'lgbtq':       "#527BCD",  
    }
    # Assign colours; fallback to grey for unknown labels
    colors = [color_map.get(lbl, 'grey') for lbl in labels]

    # Create compact, high-resolution figure
    plt.figure(figsize=(5.5, 4), dpi=155)

    # Draw pie chart with formatted percentages
    wedges, texts, autotexts = plt.pie(
        sizes,
        labels=None,
        autopct='%1.1f%%',
        pctdistance=0.55,
        startangle=90,
        colors=colors,
        wedgeprops={'linewidth': 2, 'edgecolor': 'white'}
    )

    # Style displayed percentage numbers
    for t in autotexts:
        t.set_fontsize(10)
        t.set_color("black")

    # Title
    plt.title(f"Social Group Distribution: {name}", fontsize=16)

    # Legend placed to the right of the figure
    plt.legend(
        wedges,
        labels,
        title="Domain",
        loc="center left",
        bbox_to_anchor=(1.02, 0.5),
        fontsize=11,
        title_fontsize=12
    )

    plt.tight_layout()
    plt.show()

In [ ]:
def format_text(texts: pd.Series) -> pd.Series:
    
    """
    Normalise Russian stereotype strings.

    Operations
    ----------
    - remove the phrases ", как и все люди," and ", как и все остальные,"
    - lowercase
    - remove punctuation (including comma)
    - replace '-' and '—' with spaces
    - collapse multiple spaces
    - normalise 'ё' → 'е'

    Parameters
    ----------
    texts : pd.Series
        Series of raw text strings.

    Returns
    -------
    pd.Series
        Normalised text strings.
    """

    # remove all punctuation except underscore (keep '_' if you use it as a token)
    punc = ''.join(ch for ch in string.punctuation if ch not in '_')

    # replace '-' and '—' with spaces, remove other punctuation (including commas)
    trans_table = str.maketrans('-—', '  ', punc)

    PHRASES_TO_REMOVE = [
        "как и все люди",
        "как и все остальные",
        "как и другие люди",
        "как и другие народы",
        "как и другие нации" 
    ]

    def _norm(s: str) -> str:
        s = str(s)

        # remove specific filler phrases
        for ph in PHRASES_TO_REMOVE:
            s = s.replace(ph, "")

        # lowercase + strip punctuation
        s = s.lower().translate(trans_table)

        # collapse multiple spaces
        s = " ".join(s.split())

        # normalise ё → е
        s = s.replace("ё", "е")

        return s

    return texts.apply(_norm)

In [ ]:
def clean_rubist(df: pd.DataFrame, text_col: str = "text") -> pd.DataFrame:
    
    """
    Clean and restructure the augmented RUBIAS dataset.

    Steps:
    - Drop rows where stereotype_type is missing
    - Drop old 'category' column if present
    - Rename 'label_level' → 'category'
    - Create 'label' column: category_stereotype_type
    - Reorder columns to: stereotype_type, text, category, label
    """

    df = df.copy()

    # Drop rows with missing stereotype_type
    df = df.dropna(subset=["stereotype_type"])

    # Drop old category column if exists
    if "category" in df.columns:
        df = df.drop(columns=["category"])

    # Rename label_level → category
    df = df.rename(columns={"label_level": "category"})
    
    # Format strings
    df[text_col] = format_text(df[text_col])
    
    # Remove duplicates
    df = df.drop_duplicates(subset="text")

    # Reorder columns
    desired_order = ["stereotype_type", text_col, "category"]
    df = df[desired_order]

    return df

In [ ]:
def _grouped_barplot(percent_table, title, color_map):
    
    """
    Plot a grouped bar chart with thin bars and spacing.
    Clean style, no y-label, no legend title.
    """

    categories = percent_table.index.tolist()
    labels = percent_table.columns.tolist()

    x = np.arange(len(categories))

    total_width = 0.55
    width = total_width / max(len(labels), 1)

    fig, ax = plt.subplots(figsize=(5, 3.5), dpi=200)

    # Bars
    for i, lab in enumerate(labels):
        offsets = x - total_width/2 + (i + 0.5) * width
        ax.bar(
            offsets,
            percent_table[lab].values,
            width=width * 0.75,
            label=lab,
            color=color_map.get(lab, "grey"),
            edgecolor="black",
            linewidth=0.5,
        )

    # Axis Style
    ax.set_facecolor("white")
    for spine in ax.spines.values():
        spine.set_visible(True)

    ax.set_xticks(x)
    ax.set_xticklabels(categories, fontsize=11)

    ax.set_ylim(0, 100)
    ax.set_yticks(np.arange(0, 101, 10))
    ax.tick_params(axis='y', labelsize=9)
    ax.set_ylabel("")  # removed y-label

    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda v, pos: f"{int(v)}%"))

    ax.set_title(title, fontsize=15, pad=12)

    ax.yaxis.grid(True, linestyle='-', alpha=0.13)

    ax.legend(
        loc="upper center",
        bbox_to_anchor=(0.5, -0.15),
        ncol=len(labels),
        frameon=False,
        fontsize=10
    )

    fig.tight_layout()
    plt.show()

In [ ]:
def plot_sentiment_by_category(df, category_col="category", sentiment_col="sentiment"):

    tab = pd.crosstab(df[category_col], df[sentiment_col], normalize="index") * 100

    tab = tab.reindex(
        index=["stereotype", "neutral", "unrelated"],
        columns=["positive", "neutral", "negative"]
    ).fillna(0)

    _grouped_barplot(tab, "Sentiment Classifications by Category (RuBIST)", SENTIMENT_COLORS)

In [ ]:
def plot_toxic_by_category(df, category_col="category", toxic_col="regard"):

    tab = pd.crosstab(df[category_col], df[toxic_col], normalize="index") * 100

    tab = tab.reindex(
        index=["stereotype", "neutral", "unrelated"],
        columns=["toxic", "non_toxic"]
    ).fillna(0)

    _grouped_barplot(tab, "Toxicity Classifications by Category (RuBIST)", TOXIC_COLORS)

In [ ]:
def plot_sentiment_for_stereotypes_by_group(df,
                                            category_col="category",
                                            group_col="stereotype_type",
                                            sentiment_col="sentiment"):

    subset = df[df[category_col] == "stereotype"]

    tab = pd.crosstab(subset[group_col], subset[sentiment_col], normalize="index") * 100

    tab = tab.reindex(
        index=["gender", "profession", "nationality", "lgbtq"],
        columns=["positive", "neutral", "negative"]
    ).fillna(0)

    _grouped_barplot(
        tab,
        "Proportion of Sentiment Classifications\nfor Stereotypical Sentences - By Group",
        SENTIMENT_COLORS
    )

In [ ]:
def plot_toxic_for_stereotypes_by_group(df,
                                        category_col="category",
                                        group_col="stereotype_type",
                                        toxic_col="regard"):

    subset = df[df[category_col] == "stereotype"]

    tab = pd.crosstab(subset[group_col], subset[toxic_col], normalize="index") * 100

    tab = tab.reindex(
        index=["gender", "profession", "nationality", "lgbtq"],
        columns=["toxic", "non_toxic"]
    ).fillna(0)

    _grouped_barplot(
        tab,
        "Proportion of Toxicity Classifications\nfor Stereotypical Sentences - By Group",
        TOXIC_COLORS
    )

In [ ]:
def drop_semantic_duplicates(
    df: pd.DataFrame,
    text_col: str = "text",
    group_col: str = "stereotype_type",
    model_name: str = "DeepPavlov/rubert-base-cased-sentence",
    border_sim: float = 0.98,
):
    
    """
    Remove semantically near-duplicate text entries from a dataframe.

    This function computes sentence embeddings using a SentenceTransformer
    model and identifies near-duplicate sentences based on cosine similarity.
    Only sentences belonging to the same group (e.g., same stereotype type)
    are compared. For each pair of sentences that exceed the similarity 
    threshold, the later-indexed entry is removed. Detected duplicates 
    are printed to stdout.

    Parameters
    ----------
    df : pandas.DataFrame
        Input dataframe containing at least the text column and optionally a 
        grouping column.
    text_col : str, default "text"
        Name of the column containing raw text to evaluate for duplicates.
    group_col : str, default "stereotype_type"
        Column name determining groups within which similarity comparisons 
        are performed. Sentences from different groups are never compared.
    model_name : str, default "DeepPavlov/rubert-base-cased-sentence"
        Identifier of a SentenceTransformer model used to compute embeddings.
    border_sim : float, default 0.98
        Cosine similarity threshold above which two sentences are considered
        near-duplicates. Must be in the range [0, 1].

    Returns
    -------
    pandas.DataFrame
        A cleaned dataframe with near-duplicate rows removed and the index
        reset.

    Notes
    -----
    - The function prints each detected near-duplicate pair, including the
      kept sentence, removed sentence, and similarity score.
    - Duplicate detection is greedy: the earliest occurrence is preserved,
      and any later duplicates are removed.
    - Performance may degrade for very large datasets due to O(n^2)
      pairwise similarity comparisons.

    Examples
    --------
    >>> df_clean = drop_semantic_duplicates(
    ...     df,
    ...     text_col="text",
    ...     group_col="stereotype_type",
    ...     border_sim=0.90,
    ... )
    >>> df_clean.head()
    """
    
    df = df.reset_index(drop=True).copy()

    sent_encoder = SentenceTransformer(model_name)
    texts = df[text_col].tolist()
    embeddings = sent_encoder.encode(texts, convert_to_tensor=True)

    to_remove = set()
    n = len(df)

    for i in range(n):
        if i in to_remove:
            continue
        for j in range(i + 1, n):
            if j in to_remove:
                continue

            if df.loc[i, group_col] != df.loc[j, group_col]:
                continue

            sim = util.pytorch_cos_sim(embeddings[i], embeddings[j]).item()

            if sim > border_sim:
                print("-" * 80)
                print(f"Duplicates Found (Similarity = {sim:.3f})")
                print(f"Saved [{i}]: {df.loc[i, text_col]}")
                print(f"Removed [{j}]: {df.loc[j, text_col]}")
                print("-" * 80)

                to_remove.add(j)

    print(f"\nTotal near-duplicates removed: {len(to_remove)}\n")

    return df.drop(index=list(to_remove)).reset_index(drop=True)

### $\color{pink}{Question\ 1:}$ Justify architectural modifications for new context

#### Data Preparation

In [ ]:
# Load files
rubist_aug = pd.read_csv("COMP0173_Temp_Data/rubist_aug.csv", encoding="utf-8")
rubist_aug_second= pd.read_csv("COMP0173_Temp_Data/rubist_aug_second.csv", encoding="utf-8")

In [ ]:
# Clean dataset
rubist_aug = clean_rubist(rubist_aug)
rubist_aug.head()
# Save cleaned final version
rubist_aug.to_csv("COMP0173_Data/rubist.csv", index=False)

In [ ]:
# Clean dataset
rubist_aug_second = clean_rubist(rubist_aug_second)
rubist_aug_second.head()
# Save cleaned final version
rubist_aug_second.to_csv("COMP0173_Data/rubist_second.csv", index=False)

In [ ]:
# Load final version 
rubist = pd.read_csv("COMP0173_Data/rubist.csv", encoding="utf-8")
rubist_second = pd.read_csv("COMP0173_Data/rubist_second.csv", encoding="utf-8")

In [ ]:
# Run EDA
target_dist = prepare_target_variable_distribution(rubist, "category")
group_dist = prepare_group_distribution(rubist, "stereotype_type")
text_length = prepare_text_length_analysis(rubist, "text")
create_word_cloud(rubist, text_col='text', output_filename='rubist_wordcloud.png')

# Run EDA
target_dist = prepare_target_variable_distribution(rubist_second, "category")
group_dist = prepare_group_distribution(rubist_second, "stereotype_type")
text_length = prepare_text_length_analysis(rubist_second, "text")
create_word_cloud(rubist_second, text_col='text', output_filename='rubist_second_wordcloud.png')

In [ ]:
# Sentiment and Toxicity
rubist_sentiment = analyse_sentiment_and_regard(rubist, text_col="text")
rubist_sentiment.head()
rubist_sentiment.to_csv("COMP0173_Results/rubist_sentiment", index=False, encoding="utf-8-sig")

# Sentiment and Toxicity
rubist_second_sentiment = analyse_sentiment_and_regard(rubist, text_col="text")
rubist_second_sentiment.head()
rubist_second_sentiment.to_csv("COMP0173_Results/rubist_second_sentiment", index=False, encoding="utf-8-sig")

In [ ]:
rubist_sentiment = pd.read_csv("COMP0173_Results/rubist_sentiment", encoding="utf-8-sig")
rubist_second_sentiment = pd.read_csv("COMP0173_Results/rubist_second_sentiment", encoding="utf-8-sig")

In [ ]:
# Colour maps
SENTIMENT_COLORS = {
    "positive": "#559A67",  
    "neutral":  "#F1A72F",  
    "negative": "#CA5353", 
}

TOXIC_COLORS = {
    "toxic":      "#CA5353",
    "non_toxic":  "#559A67",
}

In [ ]:
# Generate all four plots
plot_sentiment_by_category(rubist_sentiment)
plot_toxic_by_category(rubist_sentiment)
plot_sentiment_for_stereotypes_by_group(rubist_sentiment)
plot_toxic_for_stereotypes_by_group(rubist_sentiment)

In [ ]:
# Generate all four plots
plot_sentiment_by_category(rubist_second_sentiment)
plot_toxic_by_category(rubist_second_sentiment)
plot_sentiment_for_stereotypes_by_group(rubist_second_sentiment)
plot_toxic_for_stereotypes_by_group(rubist_second_sentiment)

#### Train models - Logistic Regression (Spacy Russian)

In [ ]:
from Logistic_Regression_Russian import (data_loader, train_model, evaluate_model)

gc.collect()
torch.cuda.empty_cache()

# Load and combine relevant datasets
train_data_rubist, test_data_rubist = data_loader(csv_file_path='COMP0173_Data/rubist.csv', labelling_criteria='stereotype', dataset_name='rubist', sample_size=1000000, num_examples=5)
train_data_rubist_second, test_data_rubist_second = data_loader(csv_file_path='COMP0173_Data/rubist_second.csv', labelling_criteria='stereotype', dataset_name='rubist_second', sample_size=1000000, num_examples=5)


# Execute full pipeline for logistic regression tfidf model
train_model(train_data_rubist, model_output_base_dir='model_output_LR_tfidf', dataset_name='rubist_trained', feature_type='tfidf', seed=42)
evaluate_model(test_data_rubist, model_output_dir='model_output_LR_tfidf/rubist_trained', result_output_base_dir='result_output_LR_tfidf', dataset_name='rubist', feature_type='tfidf', seed=42)

gc.collect()
torch.cuda.empty_cache()

train_model(train_data_rubist_second, model_output_base_dir='model_output_LR_tfidf', dataset_name='rubist_second_trained', feature_type='tfidf', seed=42)
evaluate_model(test_data_rubist_second, model_output_dir='model_output_LR_tfidf/rubist_second_trained', result_output_base_dir='result_output_LR_tfidf', dataset_name='rubist_second', feature_type='tfidf', seed=42)

gc.collect()
torch.cuda.empty_cache()

# Execute full pipeline for logistic regression embedding model
train_model(train_data_rubist, model_output_base_dir='model_output_LR_embedding', dataset_name='rubist_trained', feature_type='embedding', seed=42)
evaluate_model(test_data_rubist, model_output_dir='model_output_LR_embedding/rubist_trained', result_output_base_dir='result_output_LR_embedding', dataset_name='rubist', feature_type='embedding', seed=42)

gc.collect()
torch.cuda.empty_cache()

train_model(train_data_rubist_second, model_output_base_dir='model_output_LR_embedding', dataset_name='rubist_second_trained', feature_type='embedding', seed=42)
evaluate_model(test_data_rubist_second, model_output_dir='model_output_LR_embedding/rubist_second_trained', result_output_base_dir='result_output_LR_embedding', dataset_name='rubist_second', feature_type='embedding', seed=42)

ModuleNotFoundError: No module named 'Logistic_Regression_Russian'

#### Train models - DeepPavlov_rubert

In [ ]:
import os
os.environ["HF_HOME"] = "/tmp/hf"
os.environ["TRANSFORMERS_CACHE"] = "/tmp/hf"
os.makedirs("/tmp/hf", exist_ok=True)

In [ ]:
from BERT_Models_Fine_Tuning_Russian import (data_loader, train_model, evaluate_model)

gc.collect()
torch.cuda.empty_cache()

# Load and combine relevant datasets
train_data_rubist, test_data_rubist = data_loader(csv_file_path='COMP0173_Data/rubist.csv', labelling_criteria='stereotype', dataset_name='rubist', sample_size=1000000, num_examples=5)
train_data_rubist_second, test_data_rubist_second = data_loader(csv_file_path='COMP0173_Data/rubist_second.csv', labelling_criteria='stereotype', dataset_name='rubist_second', sample_size=1000000, num_examples=5)

# Execute full pipeline for Deepavlov model
train_model(train_data_rubist, model_path='DeepPavlov/rubert-base-cased', batch_size=64, epoch=6, learning_rate=2e-5, model_output_base_dir='model_output_deeppavlov_rubert', dataset_name='rubist_trained', seed=42)
evaluate_model(test_data_rubist, model_output_dir='model_output_deeppavlov_rubert/rubist_trained', result_output_base_dir='result_output_deeppavlov_rubert', dataset_name='rubist_trained', seed=42)

gc.collect()
torch.cuda.empty_cache()

train_model(train_data_rubist_second, model_path='DeepPavlov/rubert-base-cased', batch_size=64, epoch=6, learning_rate=2e-5, model_output_base_dir='model_output_deeppavlov_rubert', dataset_name='rubist_second_trained', seed=42)
evaluate_model(test_data_rubist_second, model_output_dir='model_output_deeppavlov_rubert/rubist_second_trained', result_output_base_dir='result_output_deeppavlov_rubert', dataset_name='rubist_second_trained', seed=42)

#### Train models - roberta_base

In [ ]:
from BERT_Models_Fine_Tuning_Russian import (data_loader, train_model, evaluate_model)

# Load and combine relevant datasets
train_data_rubist, test_data_rubist = data_loader(csv_file_path='COMP0173_Data/rubist.csv', labelling_criteria='stereotype', dataset_name='rubist', sample_size=1000000, num_examples=5)
train_data_rubist_second, test_data_rubist_second = data_loader(csv_file_path='COMP0173_Data/rubist_second.csv', labelling_criteria='stereotype', dataset_name='rubist_second', sample_size=1000000, num_examples=5)

# Execute full pipeline for Deepavlov model
train_model(train_data_rubist, model_path='ai-forever/ruBert-base', batch_size=64, epoch=6, learning_rate=2e-5, model_output_base_dir='model_output_ruberta_base', dataset_name='rubist_trained', seed=42)
evaluate_model(test_data_rubist, model_output_dir='model_output_ruberta_base/rubist_trained', result_output_base_dir='result_output_ruberta_base', dataset_name='rubist_trained', seed=42)

train_model(train_data_rubist_second, model_path='ai-forever/ruBert-base', batch_size=64, epoch=6, learning_rate=2e-5, model_output_base_dir='model_output_ruberta_base', dataset_name='rubist_second_trained', seed=42)
evaluate_model(test_data_rubist_second, model_output_dir='model_output_ruberta_base/rubist_second_trained', result_output_base_dir='result_output_ruberta_base', dataset_name='rubist_second_trained', seed=42)

#### Train models - FacebookAI/xlm-roberta-base

In [ ]:
from BERT_Models_Fine_Tuning_Russian import (data_loader, train_model, evaluate_model)

# Load and combine relevant datasets
train_data_rubist, test_data_rubist = data_loader(csv_file_path='COMP0173_Data/rubist.csv', labelling_criteria='stereotype', dataset_name='rubist', sample_size=1000000, num_examples=5)
train_data_rubist_second, test_data_rubist_second = data_loader(csv_file_path='COMP0173_Data/rubist_second.csv', labelling_criteria='stereotype', dataset_name='rubist_second', sample_size=1000000, num_examples=5)

# Execute full pipeline for Deepavlov model
train_model(train_data_rubist, model_path='FacebookAI/xlm-roberta-base', batch_size=64, epoch=6, learning_rate=2e-5, model_output_base_dir='model_output_xlm_roberta_base', dataset_name='rubist_trained', seed=42)
evaluate_model(test_data_rubist, model_output_dir='model_output_xlm_roberta_base/rubist_trained', result_output_base_dir='result_output_xlm_roberta_base', dataset_name='rubist_trained', seed=42)

train_model(train_data_rubist_second, model_path='FacebookAI/xlm-roberta-base', batch_size=64, epoch=6, learning_rate=2e-5, model_output_base_dir='model_output_xlm_roberta_base', dataset_name='rubist_second_trained', seed=42)
evaluate_model(test_data_rubist_second, model_output_dir='model_output_xlm_roberta_base/rubist_second_trained', result_output_base_dir='result_output_xlm_roberta_base', dataset_name='rubist_second_trained', seed=42)

### $\color{pink}{Question\ 2:}$ Document hyperparameter tuning process

## Part 5: Evaluate the adapted model, comparing performance metrics with the original study

### $\color{pink}{Question\ 1:}$ Compare original vs. adapted model performance

### $\color{pink}{Question\ 2:}$ Use appropriate metrics for problem type

### $\color{pink}{Question\ 3:}$ Conduct statistical significance testing

### $\color{pink}{Question\ 4:}$ Analyze failure cases

## References 

[1] Theo King, Zekun Wu, Adriano Koshiyama, Emre Kazim, and Philip Treleaven. 2024.
HEARTS: A holistic framework for explainable, sustainable and robust text stereotype detection.
arXiv preprint arXiv:2409.11579.
Available at: https://arxiv.org/abs/2409.11579
(Accessed: 4 December 2025).
https://doi.org/10.48550/arXiv.2409.11579

[2] Theo King, Zekun Wu, Adriano Koshiyama, Emre Kazim, and Philip Treleaven. 2024.
HEARTS-Text-Stereotype-Detection (GitHub Repository).
Available at: https://github.com/holistic-ai/HEARTS-Text-Stereotype-Detection
(Accessed: 4 December 2025).

[3] Theo King, Zekun Wu, Adriano Koshiyama, Emre Kazim, and Philip Treleaven. Holistic AI. 2024.
EMGSD: Expanded Multi-Group Stereotype Dataset (HuggingFace Dataset).
Available at: https://huggingface.co/datasets/holistic-ai/EMGSD
(Accessed: 4 December 2025).

[4] University College London Technical Support Group (TSG).
2025. GPU Access and Usage Documentation.
Available at: https://tsg.cs.ucl.ac.uk/gpus/
(Accessed: 6 December 2025).

[5] United Nations. 2025. The 2030 Agenda for Sustainable Development. 
Available at: https://sdgs.un.org/2030agenda 
(Accessed: 6 December 2025).

[] Natasha. Russian NLP Library (conda-forge distribution). Available at:
https://anaconda.org/conda-forge/natasha
(Accessed 8 December 2025).

[] Anthropic. Claude Artifact. Available at:
https://claude.ai/public/artifacts/ab5532d8-7d61-4a98-acec-5cc4236f0d74
(Accessed: 8 December 2025).

## References: Data Collection

Gender Stereotypes

https://adme.media/articles/11-stereotipov-o-muzhchinah-i-zhenschinah-kotorye-davno-ustareli-no-mnogie-s-nimi-tak-i-zhivut-2526726/

https://t-j.ru/gender-stereotypes-cases/?utm_referrer=https%3A%2F%2Fwww.google.com%2F

https://klinikaexpert.ru/articles/v-yarlykah-lozhnye-stereotipy-o-muzhchinah-i-zhenschinah

https://news.zerkalo.io/life/54154.html

https://www.sbras.info/articles/editors/gendernye-stereotipy-v-kotorye-pora-perestat-verit

https://www.rbc.ua/ukr/styler/rozpovsyudzheni-ta-zastarili-stereotipi-cholovikiv-1709482914.html

https://burninghut.ru/stereotipy-o-muzhchinakh-i-zhenshhinakh/

https://www.sravni.ru/text/5-stereotipov-o-muzhchinakh-i-zhenshhinakh-kotorye-plokho-vlijajut-na-finansy-semi/

Profession Stereotypes

https://psy.1sept.ru/article.php?ID=200301712

https://peopletalk.ru/article/10-samyh-populyarnyh-stereotipov-o-professiyah-kotorye-besyat/

https://adukar.com/by/news/abiturientu/stereotipy-o-professiyah

https://kemgmu.ru/about_the_university/news/11672/

https://dzen.ru/a/YZ81OX7HcALiVr1k

https://nafi.ru/en/analytics/samye-rasprostranennye-stereotipy-rossiyan-ob-it-professiyakh-/

https://mosdigitals.ru/blog/pochemu-ne-lyubyat-yuristov-osnovnye-prichiny-i-stereotipy

https://medium.com/juris-prudence/%D1%82%D1%80%D0%B0%D0%B4%D0%B8%D1%86%D0%B8%D0%B8-%D1%8E%D1%80%D0%B8%D1%81%D1%82%D0%BE%D0%B2-%D0%BF%D1%80%D0%B0%D0%B2%D0%B4%D0%B0-%D0%B2%D1%8B%D0%BC%D1%8B%D1%81%D0%B5%D0%BB-%D0%B1%D1%83%D0%BB%D1%88%D0%B8%D1%82-cc3de98cdfec

https://stereotypes_actors.tilda.ws/

https://m.vk.com/wall-181816199_1644

https://kovrov.ecvdo.ru/states/stereotipy-o-professii-ekonomista-chto-pravda-a-chto-vymysel

https://urzhum.ecvdo.ru/states/razvenchivaem-mify-o-professii-finansista

https://azov.ecvdo.ru/states/mify-o-populyarnyh-professiyah?utm_referrer=https%3A%2F%2Fwww.google.com%2F

https://igrim.ecvdo.ru/states/mify-i-pravda-o-rabote-v-sfere-obrazovaniya?utm_referrer=https%3A%2F%2Fwww.google.com%2F

https://brodude.ru/8-stereotipov-o-rabote-shef-povara/

https://www.maximonline.ru/lifestyle/7-glavnykh-mifov-o-rabote-bortprovodnikov-id6443401/

https://www.sports.ru/football/blogs/477244.html

https://dnmu.edu.ua/old/pro-meditsinu/3633-o-hirurgah

https://vc.ru/id3158218/1127046-mify-o-pilotah-ty-tozhe-tak-dumal

https://www.psychologies.ru/wellbeing/5-strashnyih-mifov-o-detskom-balete-v-kotoryie-pora-perestat-verit/

https://masterok.livejournal.com/11781279.html

https://adme.media/articles/10-mifov-o-balete-kotorye-kinoshniki-pridumali-radi-vau-effekta-a-my-i-kupilis-2514646/

https://mir24.tv/articles/16379683/8-glavnyh-stereotipov-o-rabote-advokata-merkantilnye-ciniki-ili-professionaly

https://omsk.ecvdo.ru/states/mify-o-dizajnerskoj-professii-razbiraemsya-chto-pravda-a-chto-net?utm_referrer=https%3A%2F%2Fwww.google.com%2F

https://media.contented.ru/vdohnovenie/kofebrejk/5-mifov-o-dizaynerah/

https://ashleyhome.am/ru/blogs/news/steriotipy-o-dizainerax

https://lifehacker.ru/7-stereotipov-o-rabote-barmena/

https://klepachsv.livejournal.com/24127.html

https://www.championat.com/lifestyle/article-4793635-razoblachaem-top-9-stereotipov-o-sportsmenah-pravda-ili-vymysel.html

https://krasotuli.com/25199-stereotipy-o-parikmaherskom-dele-razvenchivaem-mify.html

https://maycenter.ru/blog/mifi-o-professii-parikmakhera

https://m.ok.ru/group/70000000389722/topic/157405964165210?opncmnt
https://nacasting.ru/statii/mify-o-rezhisserakh

https://omsk.ecvdo.ru/states/mify-o-professii-arhitektora-chto-na-samom-dele-vazhno-dlya-uspeha-v-etoj-sfere

http://www.lookatme.ru/flow/posts/fashion-radar/181029-mify-i-realii-o-modelnom-biznese

https://rylskova.com/%D0%BC%D0%B8%D1%84%D1%8B-%D0%BE-%D0%BF%D1%80%D0%BE%D1%84%D0%B5%D1%81%D1%81%D0%B8%D0%BE%D0%BD%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D1%85-%D1%84%D0%BE%D1%82%D0%BE%D0%B3%D1%80%D0%B0%D1%84%D0%B0%D1%85/

Nationality/Race Stereotypes

https://kuban24.tv/item/10-samyh-rasprostranennyh-stereotipov-o-raznyh-natsionalnostyah

https://mir24.tv/articles/16626782/10-stereotipov-ob-indejcah:-razoblachenie-mifov-i-udivitelnye-fakty

https://tandem.net/ru/blog/russian-stereotypes-fact-fiction

https://tandem.net/ru/blog/british-stereotypes-fact-or-myth

https://meschool.ru/poleznoe/top-7-stereotipov-o-britancakh/

https://linguacats.com/ru/stati/o-chjom-molchat-frantsuzhenki

https://smapse.livejournal.com/699411.html

https://francaisclub.ru/%D1%81%D1%82%D0%B5%D1%80%D0%B5%D0%BE%D1%82%D0%B8%D0%BF%D1%8B-%D0%BE-%D1%84%D1%80%D0%B0%D0%BD%D1%86%D1%83%D0%B7%D0%B0%D1%85/

https://www.bestprivateguides.com/articles/stereotipi-ob-italyantsah-art-69.php

https://maminklub.lv/rebionok/stereotipy-ob-ispantsakh-pravda-i-lozh-623074/

https://pikabu.ru/story/stereotipyi_ob_ispantsakh__pravda_i_vyimyisel_6306872

https://chetyre-jelania.livejournal.com/197804.html

https://www.staypoland.com/ru/poland/stereotipy-o-polshe/

https://abea.com.ua/ru/top-10-pravdyvykh-stereotypov-ob-ukrayne-y-ukrayntsakh

https://vancouverok.com/15-stereotipov-o-kanadtsah-kotorye-yavlyayutsya-pravdoj/

https://nashvancouver.com/6-lozhnih-stereotipov-o-kanadcax/

https://amivisa.ru/blog/usa/stereotipy-ob-amerikancax-pravda-i-vymysel/

https://www.english-language.ru/articles/informative/stereotipyi-ob-amerikanczax/

https://tonkosti.ru/%D0%96%D1%83%D1%80%D0%BD%D0%B0%D0%BB/11_%D0%B4%D0%B8%D0%BA%D0%BE%D0%B2%D0%B0%D1%82%D1%8B%D1%85_%D1%81%D1%82%D0%B5%D1%80%D0%B5%D0%BE%D1%82%D0%B8%D0%BF%D0%BE%D0%B2_%D0%BE_%D1%80%D1%83%D1%81%D1%81%D0%BA%D0%B8%D1%85,_%D0%B2_%D0%BA%D0%BE%D1%82%D0%BE%D1%80%D1%8B%D0%B5_%D0%B2%D0%B5%D1%80%D1%8F%D1%82_%D0%B0%D0%BC%D0%B5%D1%80%D0%B8%D0%BA%D0%B0%D0%BD%D1%86%D1%8B

https://dzen.ru/a/ZfHtqQPJj3LVcVWV

https://www.reddit.com/r/AskCentralAsia/comments/ahes8h/what_are_the_stereotypes_of_the_different_central/?tl=ru

https://am.tsargrad.tv/articles/5-stereotipov-pro-armjan-i-armeniju_395939

https://adme.media/articles/ya-zhivu-v-irane-i-hochu-rasskazat-o-10-veschah-kotorye-otkroyut-etu-stranu-s-drugoj-storony-1859715/

https://chilltravel.ru/iindiastereotipi

https://www.chaochay.ru/blog/9-mifov-o-kitae-i-kitajcah

https://smapse.ru/7-banalnyh-stereotipov-o-zhitelyah-yuzhnoj-korei/

https://lifehacker.ru/stereotipy-o-severnoi-koree/

https://moya-planeta.ru/reports/view/yaponcy_lomka_stereotipov_35074

https://smapse.ru/15-stereotipov-o-yaponcah-kotorye-oni-nenavidyat/



LGBTQ+ Stereotypes

https://denis-balin.livejournal.com/329915.html

https://sojka.io/ru/guides/lgbt

http://raznoobrasije.org/wp-content/uploads/2020/07/2020_Raznoobrasije_1_Mythen-und-Fakten-u%CC%88ber-LGB.pdf

https://spherequeer.org/bisexual-week-2023/

https://gpress.info/2020/03/13/stereotipy-o-lgbt-1/

https://parniplus.com/lgbt-movement/myths-about-bisexuality/


https://www.kok.team/ru/2018-04-26/stereotipy-o-lesbiyankah

https://yvision.kz/post/gei-i-lesbiyanki-mify-i-fakty-seksualnaya-patologiya-ili-estestvennyy-process-298823

https://whatisgood.ru/theory/analytics/ulovki-lgbt-propagand/

https://holod.media/2023/05/15/myths-about-trans-people/

https://vk.com/@ovsyanart-trans-people

https://rostovgazeta.ru/news/2017-02-17/samye-rasprostranennye-mify-o-transgenderah-1353439?utm_source=google.com&utm_medium=organic&utm_campaign=google.com&utm_referrer=google.com

## References  - Stereotype

[24] Kaustubh Shivshankar Shejole and Pushpak Bhattacharyya. 2025.  
StereoDetect: Detecting Stereotypes and Anti-stereotypes the Correct Way  
Using Social Psychological Underpinnings. arXiv preprint arXiv:2504.03352.  
Available at: https://arxiv.org/abs/2504.03352  
(Accessed: 6 December 2025).

## References: RuHateBe

[6] Anna Palatkina, Elisey Rykov, Elina Sigdel, and Anna Sukhanova. 2024. 
RUHABE: Russian Hate Speech Benchmark. 
Available at: https://disk.360.yandex.ru/i/Divcpu7LaJwchw  
(Accessed: 6 December 2025).

[7] Anna Palatkina, Elisey Rykov, Elina Sigdel, and Anna Sukhanova. 2024. 
RUHABE Dataset. 
Available at: https://disk.360.yandex.ru/d/hi3PF0XuoyCRlg  
(Accessed: 6 December 2025).

[8] Anna Palatkina, Elisey Rykov, Elina Sigdel, and Anna Sukhanova. 2024. 
RUHABE Website (GitHub Repository). 
Available at: https://github.com/Annasuhstuff/RUHABE-website 
(Accessed: 6 December 2025).

## References: Russian Distorted Toxicity

[12] Alla Goryacheva. 2023. Toxicity Detection in Russian: Thesis Project Repository.  
GitHub Repository. Available at: https://github.com/alla-g/toxicity-detection-thesis/  
(Accessed: 6 December 2025).

[13] Alla Goryacheva. 2023. Russian Distorted Toxicity Corpus (TSV file).  
In *Toxicity Detection in Russian: Thesis Project Repository*.  
Available at: https://github.com/alla-g/toxicity-detection-thesis/blob/main/toxicity_corpus/russian_distorted_toxicity.tsv  
(Accessed: 6 December 2025).

## References: Kaggle - Russian Language Toxic Comments

[14] Blackmoon. 2019. Russian Language Toxic Comments Dataset.  
Kaggle. Available at: https://www.kaggle.com/datasets/blackmoon/russian-language-toxic-comments  
(Accessed: 6 December 2025).

[15] Sergey Smetanin. 2020. Toxic Comments Detection in Russian.  
In *Computational Linguistics and Intellectual Technologies: Proceedings of the International Conference “Dialogue 2020”*.  
Available at: https://doi.org/10.28995/2075-7182-2020-19-1149-1159  
(Accessed: 6 December 2025).

## References: Kaggle - Russian Hate Speech Recognition

[23] Kamil Saitov and Leon Derczynski. 2021.  
Abusive Language Recognition in Russian.  
In *Proceedings of the 8th Workshop on Balto-Slavic Natural Language Processing*,  
Kiyv, Ukraine, 20–25. Association for Computational Linguistics.  
Available at: https://aclanthology.org/2021.bsnlp-1.3/  
(Accessed: 7 December 2025).

[20] Kamil Saitov and Leon Derczynski. 2021.   
Russian Hate Speech Recognition (GitHub Repository).  
Available at: https://github.com/Sariellee/Russan-Hate-speech-Recognition 
(Accessed: 6 December 2025).

## References: Kaggle - Misc

[16] Bertie Vidgen and Leon Derczynski. 2020.  
Directions in abusive language training data, a systematic review: Garbage in, garbage out.  
*PLOS ONE*, 15, 12, e0243300.  
Available at: https://doi.org/10.1371/journal.pone.0243300  
(Accessed: 6 December 2025).

[17] Fabio Poletto, Valerio Basile, Manuela Sanguinetti, Cristina Bosco, and Viviana Patti. 2021.  
Resources and benchmark corpora for hate speech detection: A systematic review.  
*Language Resources & Evaluation*, 55, 477–523.  
Available at: https://doi.org/10.1007/s10579-020-09502-8  
(Accessed: 6 December 2025).

[18] Surendrabikram Thapa, Aditya Shah, Farhan Jafri, Usman Naseem, and Imran Razzak. 2022.  
A Multi-Modal Dataset for Hate Speech Detection on Social Media: Case-study of Russia–Ukraine Conflict.  
In *Proceedings of the 5th Workshop on Challenges and Applications of Automated Extraction of Socio-political Events from Text (CASE)*,  
1–6. Abu Dhabi, United Arab Emirates (Hybrid). Association for Computational Linguistics.  
Available at: https://aclanthology.org/2022.case-1.1  
(Accessed: 6 December 2025).

[19] Surendrabikram Thapa, Farhan Ahmad Jafri, Kritesh Rauniyar, Mehwish Nasim, and Usman Naseem. 2024.  
RUHate-MM: Identification of Hate Speech and Targets using Multimodal Data from Russia–Ukraine Crisis.  
In *Companion Proceedings of the ACM Web Conference 2024 (WWW '24)*.  
Association for Computing Machinery, New York, NY, USA, 1854–1863.  
Available at: https://doi.org/10.1145/3589335.3651973  
(Accessed: 6 December 2025).

[21] Ekaterina Pronoza, Polina Panicheva, Olessia Koltsova, and Paolo Rosso. 2021.  
Detecting ethnicity-targeted hate speech in Russian social media texts.  
Information Processing & Management, 58, 6 (2021), 102674.  
Available at: https://www.sciencedirect.com/science/article/pii/S0306457321001606  
(Accessed: 6 December 2025).  
https://doi.org/10.1016/j.ipm.2021.102674

[22] X. Wen, Y. Wang, K. Wang, and R. Sui. 2022.  
A Russian Hate Speech Corpus for Cybersecurity Applications.  
In *Proceedings of the 2022 IEEE 8th International Conference on Big Data Security on Cloud (BigDataSecurity),  
IEEE International Conference on High Performance and Smart Computing (HPSC) and  
IEEE International Conference on Intelligent Data and Security (IDS)*, Jinan, China, 41–47.  
Available at: https://doi.org/10.1109/BigDataSecurityHPSCIDS54978.2022.00018  
(Accessed: 6 December 2025).